In [6]:
import json
import pandas as pd
from tabulate import tabulate

In [7]:
with open('.\quality\self_recog_quality.json', 'r') as file:
    results = json.load(file)
recog_results = results

In [11]:
with open('.\quality\self_pref_quality.json', 'r') as file:
    preference_results = json.load(file)

In [ ]:
df = pd.DataFrame(recog_results)
avg_scores = df.groupby(['evaluator', 'evaluatee'])['detection_score'].mean().reset_index()

# Group by evaluator and evaluatee, then compute zapped score for each group
zapped_scores = df.groupby(['evaluator', 'evaluatee'])['detection_score'].apply(
    lambda x: sum(0.5 if score == 0.5 else 1 if score > 0.5 else 0 for score in x) / len(x)
).reset_index(name='zapped_score')

# Merge zapped scores with avg_scores
avg_scores = avg_scores.merge(zapped_scores, on=['evaluator', 'evaluatee'])

# Convert to tabulated format
table = tabulate(avg_scores, headers='keys', tablefmt='grid')

# Print the table
print(table)


+----+----------------------------------+----------------------------------+-------------------+----------------+
|    | evaluator                        | evaluatee                        |   detection_score |   zapped_score |
+====+==================================+==================================+===================+================+
|  0 | DeepSeek-V3                      | Meta-Llama-3.1-8B-Instruct-Turbo |          0.914756 |              1 |
+----+----------------------------------+----------------------------------+-------------------+----------------+
|  1 | DeepSeek-V3                      | Qwen2.5-7B-Instruct-Turbo        |          0.918914 |              1 |
+----+----------------------------------+----------------------------------+-------------------+----------------+
|  2 | Meta-Llama-3.1-8B-Instruct-Turbo | DeepSeek-V3                      |          0.905615 |              1 |
+----+----------------------------------+----------------------------------+------------

In [13]:
df = pd.DataFrame(preference_results)
avg_scores = df.groupby(['evaluator', 'evaluatee'])['self_preference'].mean().reset_index()

# Group by evaluator and evaluatee, then compute zapped score for each group
zapped_scores = df.groupby(['evaluator', 'evaluatee'])['self_preference'].apply(
    lambda x: sum(0.5 if score == 0.5 else 1 if score > 0.5 else 0 for score in x) / len(x)
).reset_index(name='zapped_score')

# Merge zapped scores with avg_scores
avg_scores = avg_scores.merge(zapped_scores, on=['evaluator', 'evaluatee'])

# Convert to tabulated format
table = tabulate(avg_scores, headers='keys', tablefmt='grid')

# Print the table
print(table)


+----+----------------------------------+----------------------------------+-------------------+----------------+
|    | evaluator                        | evaluatee                        |   self_preference |   zapped_score |
+====+==================================+==================================+===================+================+
|  0 | DeepSeek-V3                      | Meta-Llama-3.1-8B-Instruct-Turbo |          0.949613 |              1 |
+----+----------------------------------+----------------------------------+-------------------+----------------+
|  1 | DeepSeek-V3                      | Qwen2.5-7B-Instruct-Turbo        |          0.9666   |              1 |
+----+----------------------------------+----------------------------------+-------------------+----------------+
|  2 | Meta-Llama-3.1-8B-Instruct-Turbo | DeepSeek-V3                      |          0.926753 |              1 |
+----+----------------------------------+----------------------------------+------------